In [1]:
def remove_outliers(df, k=2):
    '''
    This function is to remove the top 25% and bottom 25% of the data for each column.
    This removes the top and bottom 50% for every column to ensure all outliers are gone.
    '''
    a=[]
    b=[]
    fences=[a, b]
    features= []
    col_list = []
    i=0
    for col in df:
            new_df=np.where(df[col].nunique()>8, True, False)
            if new_df==True:
                if df[col].dtype == 'float' or df[col].dtype == 'int':
                    '''
                    for each feature find the first and third quartile
                    '''
                    q1, q3 = df[col].quantile([.25, .75])
                    '''
                    calculate inter quartile range
                    '''
                    iqr = q3 - q1
                    '''
                    calculate the upper and lower fence
                    '''
                    upper_fence = q3 + (k * iqr)
                    lower_fence = q1 - (k * iqr)
                    '''
                    appending the upper and lower fences to lists
                    '''
                    a.append(upper_fence)
                    b.append(lower_fence)
                    '''
                    appending the feature names to a list
                    '''
                    features.append(col)
                    '''
                    assigning the fences and feature names to a dataframe
                    '''
                    var_fences= pd.DataFrame(fences, columns=features, index=['upper_fence', 'lower_fence'])
                    col_list.append(col)
                else:
                    print(col)
                    print('column is not a float or int')
            else:
                print(f'{col} column ignored')
    '''
    for loop used to remove the data deemed unecessary
    '''
    for col in col_list:
        df = df[(df[col]<= a[i]) & (df[col]>= b[i])]
        i+=1
    return df, var_fences



In [2]:
# imports

import numpy as np
import pandas as pd
import seaborn as sns
import itertools
from pydataset import data

from env import get_connection
import prepare as p

import matplotlib.pyplot as plt
%matplotlib inline

# Continuous Probability Exercises



2. Using lemonade.csv dataset and focusing on continuous variables:
        Use the IQR Range Rule and the upper and lower bounds to identify the lower outliers of each column of lemonade.csv, using the multiplier of 1.5. Do these lower outliers make sense? Which outliers should be kept?
        Use the IQR Range Rule and the upper and upper bounds to identify the upper outliers of each column of lemonade.csv, using the multiplier of 1.5. Do these upper outliers make sense? Which outliers should be kept?
        Using the multiplier of 3, IQR Range Rule, and the lower bounds, identify the outliers below the lower bound in each colum of lemonade.csv. Do these lower outliers make sense? Which outliers should be kept?
        Using the multiplier of 3, IQR Range Rule, and the upper bounds, identify the outliers above the upper_bound in each colum of lemonade.csv. Do these upper outliers make sense? Which outliers should be kept?

    Identify if any columns in lemonade.csv are normally distributed. For normally distributed columns:

        Use a 2-sigma decision rule to isolate the outliers.
            Do these make sense?
            Should certain outliers be kept or removed?

    Now use a 3-sigma decision rule to isolate the outliers in the normally distributed columns from lemonade.csv


In [3]:
# obtaining .csv

url = ('https://gist.githubusercontent.com/ryanorsinger/19bc7eccd6279661bd13307026628ace/raw/e4b5d6787015a4782f96cad6d1d62a8bdbac54c7/lemonade.csv')
    
lem = pd.read_csv(url)

In [4]:
# looking at .csv df

lem.head(3)

,Date,Day,Temperature,Rainfall,Flyers,Price,Sales
0,1/1/17,Sunday,27.0,2.00,15,0.5,10
1,1/2/17,Monday,28.9,1.33,15,0.5,13
2,1/3/17,Tuesday,34.5,1.33,27,0.5,15


In [5]:
lem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         365 non-null    object 
 1   Day          365 non-null    object 
 2   Temperature  365 non-null    float64
 3   Rainfall     365 non-null    float64
 4   Flyers       365 non-null    int64  
 5   Price        365 non-null    float64
 6   Sales        365 non-null    int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 20.1+ KB


In [6]:
# changing to 'Date' col to datetime 

lem = p.to_datetime(lem, 'Date')
lem.head()

,Day,Temperature,Rainfall,Flyers,Price,Sales,month,day
Date,,,,,,,,
2017-01-01,Sunday,27.0,2.00,15,0.5,10,1,1
2017-01-02,Monday,28.9,1.33,15,0.5,13,1,2
2017-01-03,Tuesday,34.5,1.33,27,0.5,15,1,3
2017-01-04,Wednesday,44.1,1.05,28,0.5,17,1,4
2017-01-05,Thursday,42.4,1.00,33,0.5,18,1,5


### 1. Define a function named ```get_lower_and_upper_bounds``` that has two arguments. The first argument is a pandas Series. The second argument is the multiplier, which should have a default argument of 1.5.

In [7]:
# looking at 25th & 75th quantiles

q1, q3 = lem['Flyers'].quantile([0.25, 0.75])
q1, q3

(31.0, 49.0)

In [8]:
# column IQR
    
col_iqr = q3 - q1
col_iqr

18.0

In [9]:
# finding upper and lower bounds
k = 1.5   

col_lower = q1 - k * col_iqr
col_upper = q3 + k * col_iqr

col_lower, col_upper

(4.0, 76.0)

In [10]:
lem.head()

,Day,Temperature,Rainfall,Flyers,Price,Sales,month,day
Date,,,,,,,,
2017-01-01,Sunday,27.0,2.00,15,0.5,10,1,1
2017-01-02,Monday,28.9,1.33,15,0.5,13,1,2
2017-01-03,Tuesday,34.5,1.33,27,0.5,15,1,3
2017-01-04,Wednesday,44.1,1.05,28,0.5,17,1,4
2017-01-05,Thursday,42.4,1.00,33,0.5,18,1,5


In [11]:
# upper- & lower-bounds function

def get_lower_and_upper_bounds(df, col, k = 1.5):
    
    # looking at 25th & 75th quantiles
    q1, q3 = df[col].quantile([0.25, 0.75])

        #spending_score IQR 
    col_iqr = q3 - q1

        # finding upper and lower bounds
    col_lower = q1 - k * col_iqr
    col_upper = q3 + k * col_iqr

    return (f'{col}, Q1 : {q1}, Q3 : {q3}, Column lower bound : {round(col_lower, 4)}, ' +
                 f'Column upper bound : {col_upper}')

In [12]:
# returning function 
get_lower_and_upper_bounds(lem, 'Flyers', k = 1.5)

'Flyers, Q1 : 31.0, Q3 : 49.0, Column lower bound : 4.0, Column upper bound : 76.0'

In [13]:
# function variables
q1, q3, col_lower, col_upper

(31.0, 49.0, 4.0, 76.0)

In [14]:
# upper & lower bounds for 'Rainfall'

get_lower_and_upper_bounds(lem, 'Rainfall', k = 1.5)

'Rainfall, Q1 : 0.65, Q3 : 0.91, Column lower bound : 0.26, Column upper bound : 1.3'

In [15]:
# upper, lower, quantiles for columns

lem_list = ['Temperature', 'Rainfall', 'Flyers', 'Price', 'Sales']

for col in lem_list:
    
    print(get_lower_and_upper_bounds(lem, col, k = 1.5))

Temperature, Q1 : 49.7, Q3 : 71.7, Column lower bound : 16.7, Column upper bound : 104.7
Rainfall, Q1 : 0.65, Q3 : 0.91, Column lower bound : 0.26, Column upper bound : 1.3
Flyers, Q1 : 31.0, Q3 : 49.0, Column lower bound : 4.0, Column upper bound : 76.0
Price, Q1 : 0.5, Q3 : 0.5, Column lower bound : 0.5, Column upper bound : 0.5
Sales, Q1 : 20.0, Q3 : 30.0, Column lower bound : 5.0, Column upper bound : 45.0


In [22]:
# upper bounds function

def find_out_of_upper_lower_bounds(df, col, k = 1.5):
    
    #finding the quantiles for a particular column
    q1, q3 = df[col].quantile([0.25, 0.75])
    
    # make iqr
    iqr = q3 - q1
    
    # set upper outlier boundary
    upper_bound = q3 + k * col_iqr
    
    # set lower outlier boundary
    lower_bound = q1 - k * col_iqr

    return (np.where(df[col] > upper_bound, 1, 0)), (np.where(df[col] < lower_bound, 1, 0))

### 2. Using lemonade.csv dataset and focusing on continuous variables:
        Use the IQR Range Rule and the upper and lower bounds to identify the lower outliers of each column of  lemonade.csv, using the multiplier of 1.5. Do these lower outliers make sense? Which outliers to keep ?

In [23]:
# determing whether any 'Flyers' is above the upper bound or the lower bound

find_out_of_upper_lower_bounds(lem, 'Flyers', k = 1.5)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [29]:
# assigning new columns to dataframe based on above function

lem_list = ['Temperature', 'Rainfall', 'Flyers', 'Price', 'Sales']

#def lem_upper_lower():
for col in lem_list:
    
    lem[f'{col}_upper_outliers'], lem[f'{col}_lower_outliers'] = find_out_of_upper_lower_bounds(lem, 
                                                                                                col, k = 1.5)
lem.sample(3).T    

Date,2017-05-18,2017-02-12,2017-07-14
Day,Thursday,Sunday,Friday
Temperature,72.0,55.6,92.0
Rainfall,0.67,0.83,0.5
Flyers,53,41,80
Price,0.5,0.5,0.5
Sales,30,22,40
month,5,2,7
day,18,12,14
Temperature_upper_outliers,0,0,0
Temperature_lower_outliers,0,0,0


In [25]:
# upper, lower, quantiles for columns

lem_list = ['Temperature', 'Rainfall', 'Flyers', 'Price', 'Sales']

for col in lem_list:
    
    print(get_lower_and_upper_bounds(lem, col, k = 1.5))

Temperature, Q1 : 49.7, Q3 : 71.7, Column lower bound : 16.7, Column upper bound : 104.7
Rainfall, Q1 : 0.65, Q3 : 0.91, Column lower bound : 0.26, Column upper bound : 1.3
Flyers, Q1 : 31.0, Q3 : 49.0, Column lower bound : 4.0, Column upper bound : 76.0
Price, Q1 : 0.5, Q3 : 0.5, Column lower bound : 0.5, Column upper bound : 0.5
Sales, Q1 : 20.0, Q3 : 30.0, Column lower bound : 5.0, Column upper bound : 45.0


In [20]:
lem_upper_lower()

NameError: name 'lem_upper_lower' is not defined